In [1]:
# Import packages

# Database manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud

# Natural language Toolkit packages.
# Necessary libraries and modules that are 
# going to help us do the data processing 
# from the nltk library.
import nltk

nltk.download(['punkt','stopwords'])
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string

# Regular expression
import re

# to make bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Packages to create models
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression


from sklearn.svm import LinearSVC  
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
# Model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

# Save trained models
import pickle


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\Malibongwe
[nltk_data]     Shange\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Malibongwe
[nltk_data]     Shange\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Malibongwe
[nltk_data]     Shange\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import dataset

df = pd.read_csv("train_set.csv")
df1 = pd.read_csv("test_set.csv")
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows', None)

In [4]:
df.head()

lang_id  \
0     xho   
1     xho   
2     eng   
3     nso   
4     ven   

                                                                                                                                                                                                                                                                       text  
0                                              umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika  
1              i-dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo  
2  the province of kwazulu-natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months  
3                                                 o netefatša gore o ba file dilo ka moka tše le dumelelanego ka tšona mohlala maleri a magolo a a šomišwago go fihlelela meagong e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj  
4                           khomishini ya ndinganyiso ya mbeu yo ewa maana u ya nga mulayo wa khomishini ya ndinganyiso ya mbeu u thetshelesa mbilaelo dzine dza tshimbilelana na tshialula u ya nga mbeu nahone i ivhea sa foramu ya thungo u ya nga mulayo wa ndinganyiso

In [5]:
df["lang_id"].value_counts()

ssw    3000
tso    3000
eng    3000
nso    3000
ven    3000
zul    3000
nbl    3000
xho    3000
sot    3000
afr    3000
tsn    3000
Name: lang_id, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [7]:
def clean_text(text):
    text = re.sub(r'[-!@#$(),n"%^*?:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    return text
df['cleaned_text'] = df['text'].apply(clean_text)
df1['cleaned_text'] = df1['text'].apply(clean_text)

In [8]:
df.head()

lang_id  \
0     xho   
1     xho   
2     eng   
3     nso   
4     ven   

                                                                                                                                                                                                                                                                       text  \
0                                              umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika   
1              i-dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo   
2  the province of kwazulu-natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months   
3                                                 o netefatša gore o ba file dilo ka moka tše le dumelelanego ka tšona mohlala maleri a magolo a a šomišwago go fihlelela meagong e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj   
4                           khomishini ya ndinganyiso ya mbeu yo ewa maana u ya nga mulayo wa khomishini ya ndinganyiso ya mbeu u thetshelesa mbilaelo dzine dza tshimbilelana na tshialula u ya nga mbeu nahone i ivhea sa foramu ya thungo u ya nga mulayo wa ndinganyiso   

                                                                                                                                                                                                                                                               cleaned_text  
0                                              umgaqo siseko we za amalu giselelo kumaziko axhasa ulawulo lwesi i zi ku ye  okuthath i xaxheba kwabafazi ezi ziquka phakathi kwezi ye zazo ikomisho i yokuli ga a  gokwesi i ikomisho i yamalu gelo olu tu lomza tsi afrika  
1              i dha iya kuba  obulumko bokubeka umsebe zi  aphi  a kwisebe  gokusekwe kwiimfu o zokusebe za zalo emva kokubo a a  omsebe zi ku ye oka ye ima ya o yakhe ukuba ula dulo lom tu o jalo alufa eleka ga i dha mayibize u cedo olufa elekileyo elu gelwe i layo  
2  the provi ce of kwazulu  atal departme t of tra sport i vites te ders from established co tractors experie ced i  bridge co structio  for the co structio  of the kwajolwayo tugela river pedestria  bridge  ear tugela ferry the duratio  of the project will be mo ths  
3                                                 o  etefatša gore o ba file dilo ka moka tše le dumelela ego ka tšo a mohlala maleri a magolo a a šomišwago go fihlelela meago g e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj  
4                           khomishi i ya  di ga yiso ya mbeu yo ewa maa a u ya  ga mulayo wa khomishi i ya  di ga yiso ya mbeu u thetshelesa mbilaelo dzi e dza tshimbilela a  a tshialula u ya  ga mbeu  aho e i ivhea sa foramu ya thu go u ya  ga mulayo wa  di ga yiso

In [9]:
df1.head()

index  \
0      1   
1      2   
2      3   
3      4   
4      5   

                                                                                                            text  \
0                                                     Mmasepala, fa maemo a a kgethegileng a letlelela kgato eo.   
1  Uzakwaziswa ngokufaneleko nakungafuneka eminye imitlolo engezelelako ukuqedelela ukutloliswa kwesibawo sakho.   
2                                                                     Tshivhumbeo tshi fana na ngano dza vhathu.   
3                                                   Kube inja nelikati betingevakala kutsi titsini naticocisana.   
4                                                                                  Winste op buitelandse valuta.   

                                                                                                    cleaned_text  
0                                                     mmasepala  fa maemo a a kgethegile g a letlelela kgato eo.  
1  uzakwaziswa  gokufa eleko  aku gafu eka emi ye imitlolo e gezelelako ukuqedelela ukutloliswa kwesibawo sakho.  
2                                                                     tshivhumbeo tshi fa a  a  ga o dza vhathu.  
3                                                   kube i ja  elikati beti gevakala kutsi titsi i  aticocisa a.  
4                                                                                  wi ste op buitela dse valuta.

In [10]:
X = df["cleaned_text"]
y = df["lang_id"]

In [11]:
# Split 1 : Normal data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state = 42)

In [12]:
# Declare TfidfVectorizer
TDIF_vector_1 = TfidfVectorizer(max_df=0.9, ngram_range=(1, 5), analyzer= 'char',max_features=10000)

In [13]:
LR_model_1 = LogisticRegression()
LR_model_1 = Pipeline([('tfidf' ,TDIF_vector_1), ('cfl', LogisticRegression(max_iter=1000))])

In [14]:
LR_model_1.fit(X_train, y_train)
y_pred = LR_model_1.predict(X_test)

In [15]:
f1score_4 = f1_score(y_test, y_pred, average= 'weighted')
print('f1 score:', f1score_4)

f1 score: 0.9975773317158672


In [16]:
# Classification Report
report = metrics.classification_report(y_test, y_pred)

In [17]:
print(report)

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      0.99      0.99       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       0.99      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       0.99      0.99      0.99       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [18]:
y_test = LR_model_1.predict(df1['cleaned_text'])
results = pd.DataFrame({'index' : df1['index'],'lang_id' : y_test})
results.to_csv('submission.csv', index = False)

In [19]:
results

index lang_id
0         1     tsn
1         2     nbl
2         3     ven
3         4     ssw
4         5     afr
5         6     nso
6         7     eng
7         8     sot
8         9     zul
9        10     eng
10       11     nso
11       12     tso
12       13     tsn
13       14     sot
14       15     tso
15       16     xho
16       17     xho
17       18     ssw
18       19     afr
19       20     nbl
20       21     afr
21       22     ven
22       23     zul
23       24     afr
24       25     nso
25       26     zul
26       27     ven
27       28     afr
28       29     nso
29       30     tso
30       31     zul
31       32     xho
32       33     sot
33       34     nso
34       35     ssw
35       36     afr
36       37     nso
37       38     xho
38       39     sot
39       40     eng
40       41     ssw
41       42     ssw
42       43     ven
43       44     nbl
44       45     afr
45       46     ven
46       47     zul
47       48     ven
48       49     nbl
49       50     tsn
50       51     sot
51       52     xho
52       53     sot
53       54     sot
54       55     zul
55       56     ven
56       57     nbl
57       58     xho
58       59     ssw
59       60     sot
60       61     afr
61       62     tso
62       63     afr
63       64     zul
64       65     xho
65       66     nbl
66       67     sot
67       68     nso
68       69     nso
69       70     xho
70       71     tsn
71       72     tso
72       73     xho
73       74     nso
74       75     nso
75       76     ven
76       77     zul
77       78     tso
78       79     xho
79       80     ssw
80       81     tsn
81       82     sot
82       83     xho
83       84     tsn
84       85     eng
85       86     nso
86       87     sot
87       88     tso
88       89     tso
89       90     xho
90       91     ssw
91       92     xho
92       93     zul
93       94     sot
94       95     ssw
95       96     nso
96       97     afr
97       98     ssw
98       99     ven
99      100     nbl
100     101     afr
101     102     tso
102     103     nso
103     104     zul
104     105     ven
105     106     ven
106     107     tsn
107     108     nso
108     109     sot
109     110     sot
110     111     xho
111     112     zul
112     113     xho
113     114     tso
114     115     zul
115     116     sot
116     117     afr
117     118     nso
118     119     eng
119     120     eng
120     121     sot
121     122     eng
122     123     zul
123     124     tso
124     125     nso
125     126     afr
126     127     ven
127     128     tso
128     129     tso
129     130     tsn
130     131     tso
131     132     ven
132     133     zul
133     134     sot
134     135     tsn
135     136     ssw
136     137     xho
137     138     sot
138     139     ssw
139     140     afr
140     141     sot
141     142     tsn
142     143     nbl
143     144     nso
144     145     nbl
145     146     ven
146     147     nso
147     148     sot
148     149     xho
149     150     ven
150     151     nso
151     152     xho
152     153     xho
153     154     tsn
154     155     sot
155     156     ven
156     157     ven
157     158     nso
158     159     eng
159     160     sot
160     161     zul
161     162     tso
162     163     nso
163     164     xho
164     165     zul
165     166     sot
166     167     nbl
167     168     zul
168     169     eng
169     170     nbl
170     171     xho
171     172     nso
172     173     xho
173     174     afr
174     175     tso
175     176     nso
176     177     afr
177     178     xho
178     179     tso
179     180     xho
180     181     sot
181     182     nso
182     183     ssw
183     184     ssw
184     185     sot
185     186     sot
186     187     zul
187     188     nbl
188     189     zul
189     190     nbl
190     191     xho
191     192     tso
192     193     sot
193     194     afr
194     195     zul
195     196     nso
196     197     sot
197     198     nbl
198     199     zul
199   